In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
english_texts=[]
russian_texts=[]
english_character=[]
russian_character=[]
with open("./rus.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[:30000]:
    english_text,russian_text,_=line.split("\t")
    english_texts.append(english_text)
    russian_text = "\t" + russian_text + "\n"
    russian_texts.append(russian_text)

In [3]:
for i in english_texts:
    for c in i:
        if c not in english_character:
            english_character.append(c)
            english_character.sort()

In [4]:
for j in russian_texts:
    for c in j:
        if c not in russian_character:
            russian_character.append(c)
            russian_character.sort()

In [5]:
english_d={}
for i in range(len(english_character)):
    english_d[english_character[i]]=i

In [6]:
russian_d={}
for i in range(len(russian_character)):
    russian_d[russian_character[i]]=i

In [7]:
english_encoder_tokens = len(english_character)
russian_decoder_tokens = len(russian_character)

In [8]:
max_encoder_seq_length=0
for i in english_texts:
    if len(i)>max_encoder_seq_length:
        max_encoder_seq_length=len(i)

In [9]:
max_decoder_seq_length=0
for i in russian_texts:
    if len(i)>max_decoder_seq_length:
        max_decoder_seq_length=len(i)

In [10]:
max_decoder_seq_length

60

In [11]:
encoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(english_texts[bb])
    m=0
    while m<k:
        for char in english_texts[bb][m]:
                           for i in range(len(english_character)):
                                            if english_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    encoder_input_data.append(c)

In [12]:
encoder_input_data=np.array(encoder_input_data)

In [13]:
encoder_input_data.shape

(30000, 17, 74)

In [14]:
decoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(russian_texts[bb])
    m=0
    while m<k:
        for char in russian_texts[bb][m]:
                           for i in range(len(russian_character)):
                                            if russian_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_decoder_seq_length:
        for i in range(len(russian_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_input_data.append(c)

In [15]:
decoder_input_data=np.array(decoder_input_data)

In [16]:
decoder_input_data.shape

(30000, 60, 101)

In [17]:
decoder_target_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(russian_texts[bb])
    m=1
    while m<k:
        for char in russian_texts[bb][m]:
                           for i in range(len(russian_character)):
                                            if russian_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    m=m-1
    while m<max_decoder_seq_length:
        for i in range(len(russian_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_target_data.append(c)

In [18]:
decoder_target_data=np.array(decoder_target_data)

In [19]:
decoder_target_data.shape

(30000, 60, 101)

In [20]:
batch_size = 64  # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 

In [21]:
from keras.layers import Dense
from keras.layers import LSTM,Input
from keras.models import Model

In [22]:
encoder_inputs = Input(shape=(None,len(english_character)))
encoder = LSTM(latent_dim,dropout=0.2,return_sequences=True,return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder(encoder_inputs)
encoder = LSTM(latent_dim,dropout=0.2,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_outputs_1)
encoder_states = [state_h_1,state_c_1,state_h, state_c]

In [23]:
decoder_inputs = Input(shape=(None, len(russian_character)))
decoder_lstm = LSTM(latent_dim,return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs_1, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h_1,state_c_1])
decoder_lstm_1 = LSTM(latent_dim, return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs, _, _ = decoder_lstm_1(decoder_outputs_1, initial_state=[state_h,state_c])
decoder_dense = Dense(len(russian_character), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [24]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, None, 74)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, None, 101) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, None,     │    338,944 │ input_layer[0][0] │
│                     │ 256), (None,      │            │                   │
│                     │ 256), (None,      │            │                   │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, None,     │    366,592 │ input_layer_1[0]… │
│                     │ 256), (None,      │            │ lstm[0][1],       │
│                     │ 256), (None,      │            │ lstm[0][2]        │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 256),     │    525,312 │ lstm[0][0]        │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ [(None, None,     │    525,312 │ lstm_2[0][0],     │
│                     │ 256), (None,      │            │ lstm_1[0][1],     │
│                     │ 256), (None,      │            │ lstm_1[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, None, 101) │     25,957 │ lstm_3[0][0]      │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,782,117 (6.80 MB)

 Trainable params: 1,782,117 (6.80 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.layers[2]

<LSTM name=lstm, built=True>

In [26]:
# model.compile(
#     optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
# )
# model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=batch_size,
#     epochs=epochs,
#     validation_split=0.2,
# )

In [27]:
# model.save("engtorussian.h5")

In [28]:
import keras

In [29]:
model = keras.models.load_model("engtorussian.h5")
encoder_inputs = model.input[0]  # input_1
encoder_outputs_1, state_h_enc_1, state_c_enc_1 = model.layers[2].output 
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output 
encoder_states = [state_h_enc_1, state_c_enc_1,state_h_enc, state_c_enc]
encoder_model_1 = keras.Model(encoder_inputs, encoder_states)

In [30]:
decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,),)
decoder_state_input_c = keras.Input(shape=(latent_dim,),)
decoder_state_input_h1 = Input(shape=(latent_dim,),)
decoder_state_input_c1 = Input(shape=(latent_dim,),)
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c,decoder_state_input_h1,decoder_state_input_c1]
decoder_lstm = model.layers[3]
dec_o, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
decoder_lstm_1=model.layers[5]
dec_o_1, state_h1, state_c1 = decoder_lstm_1(
    dec_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h,state_c,state_h1,state_c1]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(dec_o_1)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [31]:
reverse_input_char_index ={}
for i in range(len(english_character)):
    reverse_input_char_index[i]=english_character[i]
reverse_target_char_index ={}
for i in range(len(russian_character)):
    reverse_target_char_index[i]=russian_character[i]

In [32]:
def decode_sequence(input_seq):
    states_value=encoder_model_1.predict(input_seq)
    target_seq = np.zeros((1, 1, len(russian_character)))
    target_seq[0, 0, russian_d["\t"]] = 1.0
    flag=0
    sent=""
    while flag==0:
        output_tokens, h, c,h1,c1 = decoder_model.predict([target_seq] + states_value)
        sample = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sample]
        sent+=sampled_char
        if sampled_char == "\n" or len(sent) > max_decoder_seq_length:
            flag=1
        target_seq = np.zeros((1, 1, len(russian_character)))
        target_seq[0, 0,sample] = 1.0
        states_value = [h, c,h1,c1]
    return sent

In [1]:
english='run'

In [3]:
k=len(english)
m=0
a=[]
b=[]
c=[]
inpu=[]
while m<k:
    for char in english[m]:
        for i in range(len(english_character)):
            if english_d[char]==i:
                a.append(1)
            else:
                a.append(0)
    for kp in a:
        b.append(kp)
    c.append(b)
    b=[]
    a=[]
    m=m+1
while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
inpu.append(c)

NameError: name 'english_character' is not defined

In [59]:
inpu=np.array(inpu)

In [60]:
inpu

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]])

In [61]:
d=decode_sequence(inpu)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [62]:
print(d)

Он любит кофе.

